Archivo de Jupyter para cálculo simbólico de elementos de matriz

In [1]:
# Computación numérica
import numpy as np
from scipy.special import erf, factorial, gammainc
# Computación simbólica
from sympy import symbols, simplify, integrate, oo, E
from sympy.vector import CoordSys3D, laplacian

In [2]:
A = CoordSys3D('A', transformation='spherical') # Sistema coordenado esférico, centrado en A
B = CoordSys3D('b', transformation='spherical') # Sistema coordenado esférico, centrado en B
a, b = symbols('a b', positive=True) # exponentes orbitales Gaussianos

En general un elemento de matriz $f_{pq}$ será dado por
$$ f_{pq} = \int \chi_p^{*} (\hat{f}_1 + \hat{f}_2) \chi_q \, \mathrm{d}\mathbf{x}
    = \int \psi_p^{*} (\hat{f}_1 + \hat{f}_2) \psi_q \, \mathrm{d}\mathbf{r} \int Y_{00}^2 \, \mathrm{d}\Omega \int \sigma_p^{*}(\omega) \sigma_q(\omega) \, \mathrm{d}\omega
$$

La parte angular es 1 y la parte espinorial se trata de una delta de Kronecker, por lo cual solo los elementos de matriz $f_{pq}$ tal que sus partes espinoriales coincidan serán diferentes de cero. Las funciones radiales $\psi$ son dadas por $\psi = \varphi_A \pm \varphi_B$ donde $\varphi_C$ se trata de una $k$-contracción Gaussiana $1\mathrm{s}$ centrada en $C$ de la forma:

$$ \varphi_C = \sum_{i=1}^{k} d_i \phi_i^{(C)} \,; \qquad
\phi_i^{(C)} = c_i \exp(-\alpha_i |\mathbf{r} - \mathbf{R}_C|^2) = c_i \exp(-\alpha_i r_C^2)
$$

donde $c_i$ es el factor de normalización de la función Gaussiana $1\mathrm{s}$ :

$$ c_i = \frac{2(2\alpha_i)^{3/4}}{\pi^{1/4}} \,.$$

<div class='alert alert-block alert-warning'>
    Dado que las constantes $c_i$ no dependen más que en los coeficientes orbitales Gaussianos $\alpha_i$ estos serán añadidos en los cálculos hasta el final.
</div>

Desarrollando la parte radial donde $\mathrm{d}\mathbf{r} = r^2 \mathrm{d}r$ se obtiene
$$
\int \psi_p (\hat{f}_1 + \hat{f}_2) \psi_q \, \mathrm{d}\mathbf{r}
    = (\varphi_A | \hat{f}_1 | \varphi_A) \pm_p (\varphi_B | \hat{f}_1 | \varphi_A) \pm_q (\varphi_A | \hat{f}_1 | \varphi_B) \pm_p \pm_q (\varphi_B | \hat{f}_1 | \varphi_B)
    + (\varphi_A | \hat{f}_2 | \varphi_A) \pm_p (\varphi_B | \hat{f}_2 | \varphi_A) \pm_q (\varphi_A | \hat{f}_2 | \varphi_B) \pm_p \pm_q (\varphi_B | \hat{f}_2 | \varphi_B)
$$

Por simplicidad, se empleará la convención de notación $f_{i}^{(CD)} = (\varphi_C | \hat{f}_i | \varphi_D)$ donde $i \in \{1,2\}$ y $C, D \in \{A,B\}$. Esto para cada uno de los términos del elemento de matriz. En cuyo caso, el elemento de matriz $f_{pq}$ luciría como

$$ f_{pq} =
    f_{1}^{(AA)} \pm_p f_{1}^{(BA)} \pm_q f_{1}^{(AB)} \pm_p \pm_q f_{1}^{(BB)}
    + f_{2}^{(AA)} \pm_p f_{2}^{(BA)} \pm_q f_{2}^{(AB)} \pm_p \pm_q f_{2}^{(BB)}
$$

Como ambos núcleos son hidrógenos y espacialmente las funciones $\varphi_A$ y $\varphi_B$ son iguales, es posible garantizar que $f_{1}^{(AA)} = f_{1}^{(BB)}$, $f_{1}^{(AB)} = f_{1}^{(BA)}$, $f_{2}^{(AA)} = f_{2}^{(BB)}$ y $f_{2}^{(AB)} = f_{2}^{(BA)}$, así pues el elemento de matriz $f_{pq}$ será:

$$ f_{pq} = 2\delta_{pq} \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right] $$

Entonces los elementos de matriz fuera de la diagonal $f_{pq}$, aunque $p$ y $q$ sean ambos pares o nones, son cero. Es decir, solo los elementos en la diagonal son diferentes de cero.

$$ f_{pp} = 2 \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right] $$

In [3]:
def const(a:float):
    """
        Constante de normalización de una función tipo Gaussiana 1s
    a : exponente orbital Gaussiano
    """
    return 2*np.power(2*a, 0.75) / np.power(np.pi, 0.25)

<div class='alert alert-block alert-info'><strong>Término $f_{1}^{(AA)}$</strong></div>

El término $f_{1}^{(AA)}$ puede ser calculado facilidad:

$$ f_{1}^{(AA)}
= (\varphi_A | \hat{f}_1 | \varphi_A)
= \int \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \hat{f}_1 \left( \sum_j^{k} d_j \phi_{j}^{(A)} \right) \mathrm{d}\mathbf{r}
= \sum_i^{k} \sum_j^{k} d_i d_j \int \phi_{i}^{(A)} \hat{f}_1 \phi_{j}^{(A)} \mathrm{d}\mathbf{r}
$$

Para los términos $\hat{f}_1 \phi_{j}^{(A)}$ se tiene que

$$ \hat{f}_1 \phi_{j}^{(A)}
= \left( -\frac{1}{2} \nabla^2 \right) \exp(-\alpha_j r^2)
= -\frac{1}{2} \frac{1}{r^2} \frac{\partial}{\partial r} \left( r^2 \frac{\partial}{\partial r}\exp\left[-\alpha_j r^2 \right] \right)
= \alpha_j (3 - 2 \alpha_j r^2) \exp(-\alpha_j r^2)
= \alpha_j (3 - 2 \alpha_j r^2) \phi_{j}^{(A)}
$$

In [4]:
# corroboración trabajo previo
lapAA = simplify((-1/2) * laplacian(E**(-b*(A.r**2))))
lapAA

b*(-2.0*A.r**2*b + 3.0)*exp(-A.r**2*b)

In [5]:
ind_f1AA = simplify((A.r**2) * E**(-a*(A.r**2)) * lapAA) # integrando
int_f1AA = simplify(integrate(ind_f1AA, (A.r,0,oo))) # integral
int_f1AA
# Nota: Al integrando se añade el elemento radial r^2 pues SymPy no lo añade a pesar que A.r es una coordenada radial

0.75*sqrt(pi)*a*b/(a + b)**(5/2)

De este modo, el elemento de matriz $f_1^{AA}$ será dado por
$$ f_{1}^{(AA)}
= \sum_i^{k} \sum_j^{k} d_i d_j \int r^2 \phi_{i}^{(A)} \alpha_j(3 - 2 \, \alpha_j \, r^2) \phi_{j}^{(A)} \mathrm{d}r
= \frac{3}{4} \sqrt{\pi} \sum_i^{k} \sum_j^{k} d_i d_j \frac{\alpha_i \alpha_j}{(\alpha_i + \alpha_j)^{5/2}}
$$

Incluyendo los factores de normalización $c_i$ y $c_j$ :

$$ f_{1}^{(AA)}
= \frac{3}{4} \sqrt{\pi} \sum_i^{k} \sum_j^{k} \left( \frac{2(2\alpha_i)^{3/4}}{\pi^{1/4}} \right) \left( \frac{2(2\alpha_j)^{3/4}}{\pi^{1/4}} \right) d_i d_j \frac{\alpha_i \alpha_j}{(\alpha_i + \alpha_j)^{5/2}}
= \boxed{3 (2^{3/2}) \sum_i^{k} \sum_j^{k} d_i d_j \frac{(\alpha_i \alpha_j)^{7/4}}{(\alpha_i + \alpha_j)^{5/2}}}
$$

In [6]:
def f1AA(d:np.array, a:np.array):
    """
        Término f_1^AA
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    """
    k = len(d)
    suma = 0
    for i in range(k):
        for j in range(k):
            # const(a) = const. de normalización de Gaussianas 1s
            suma += d[i]*d[j]*np.power(a[i]*a[j], 1.75)/np.power(a[i]+a[j], 2.5)
    return 3*np.power(2,1.5)*suma

In [7]:
print("Se fijan todos los parámetros igual a 1.")
print("Cómputo directo:  ", f1AA([1],[1]))
print("Cómputo simbólico:", const(1)*const(1)*int_f1AA.subs({a:1, b:1}).evalf())

Se fijan todos los parámetros igual a 1.
Cómputo directo:   1.5
Cómputo simbólico: 1.50000000000000


<div class='alert alert-block alert-info'><strong>Término $f_{1}^{(AB)}$</strong></div>

Observe que $f_{1}^{(AB)}$ constará de $k^2$ términos pues

$$ f_{1}^{(AB)} = (\varphi_A | \hat{f}_1 | \varphi_B)
= \int \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \hat{f}_1 \left( \sum_j^{k} d_j \phi_{j}^{(B)} \right) \, \mathrm{d}\mathbf{r}
= \sum_i^{k} \sum_j^{k} d_i d_j \underbrace{\int \phi_{i}^{(A)} \hat{f}_1 \phi_{j}^{(B)} \, \mathrm{d}\mathbf{r}}_{I_1^{(AB)}}
$$

In [8]:
# integrando
lapAB = simplify((-1/2) * laplacian(E**(-b*(B.r**2))))
ind_f1AB = simplify(E**(-a*(A.r**2)) * lapAB) # integrando
ind_f1AB

-b*(2.0*b.r**2*b - 3.0)*exp(-A.r**2*a - b.r**2*b)

La integral es entonces

$$ \begin{align}
I_1^{(AB)}
&= 3\beta \int \exp\left(-\alpha \, r_A^2\right) \exp\left(-\beta \, r_B^2\right) \, \mathrm{d}\mathbf{r}
    - 2 \beta^2 \int r_B^2 \exp\left(-\alpha \, r_A^2\right) \exp\left(-\beta \, r_B^2\right) \, \mathrm{d}\mathbf{r} \\
&= 3\beta K_{ab} \int \exp\left(-p \, r_P^2\right) \, \mathrm{d}\mathbf{r}
    - 2 \beta^2 K_{\alpha \beta} \int \left( r_P + \frac{\alpha}{p} R_{AB} \right)^2 \exp\left(-p \, r_P^2\right) \, \mathrm{d}\mathbf{r} \\
&= 3\beta K_{ab} \frac{\sqrt{\pi}}{4 p^{3/2}}
    - 2 \beta^2 K_{\alpha \beta} \int \left( r_P + \frac{\alpha}{p} R_{AB} \right)^2 \exp\left(-p \, r_P^2\right) \, \mathrm{d}\mathbf{r}
\end{align}
$$

<div class='alert alert-block alert-warning'>
Szabo y Ostlund dan una expresión para la integral $I_{1}^{(AB)}$ sobre todo el espacio [1, pág.412], es decir consideran a $\mathrm{d}\mathbf{r} = r^2 \sin \theta \, \mathrm{d}r \, \mathrm{d}\theta \, \mathrm{d}\phi$. Para el presente trabajo ya se integró sobre las coordenadas angulares lo cual resultaba en un factor de 1 debido al armónico esférico $Y_{00}$. Por lo tanto, es necesario considerar un factor de $1/(4\pi)$ adicional en nuestro caso.
</div>

La expresión de $I_1^{(AB)}$ obtenida por Szabo y Ostlund, multiplicado por un factor de $1/(4\pi)$, al ser sustituido en el término $f_{1}^{(AB)}$ resulta en:
$$
\begin{align}
    f_{1}^{(AB)}
    &= \frac{1}{4\pi} \sum_i^{k} \sum_j^{k} d_i d_j \frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j} \left[ \frac{3 - 2\alpha_i \alpha_j}{\alpha_i + \alpha_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] \left[ \frac{\pi}{\alpha_i + \alpha_j} \right]^{3/2} \exp \left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j}|\mathbf{R}_A - \mathbf{R}_B|^2 \right] \\
    &= \frac{1}{4\sqrt{\pi}} \sum_i^{k} \sum_j^{k} d_i d_j \frac{\alpha_i \alpha_j}{(\alpha_i + \alpha_j)^{7/2}} (3 - 2\alpha_i \alpha_j) |\mathbf{R}_A - \mathbf{R}_B|^2 \exp \left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j}|\mathbf{R}_A - \mathbf{R}_B|^2 \right]
\end{align}
$$

la expresión general para $f_{1}^{(AB)}$ sin factores de normalización.

Considere el siguiente desarrollo al incluir a los factores de normalización

$$ \begin{align*}
f_{1}^{(AB)}
&= \frac{\sqrt{\pi}}{4} \sum_i^{k} \sum_j^{k} \left( \frac{2(2\alpha_i)^{3/4}}{\pi^{1/4}} \right) \left( \frac{2(2\alpha_j)^{3/4}}{\pi^{1/4}} \right)
d_i d_j \frac{\alpha_i \alpha_j}{(\alpha_i + \alpha_j)^{7/2}} (3 - 2\alpha_i \alpha_j) |\mathbf{R}_A - \mathbf{R}_B|^2 \exp \left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j}|\mathbf{R}_A - \mathbf{R}_B|^2 \right] \\
&= \boxed{2^{3/2} \sum_i^{k} \sum_j^{k}
d_i d_j \frac{(\alpha_i \alpha_j)^{7/4}}{(\alpha_i + \alpha_j)^{7/2}} (3 - 2\alpha_i \alpha_j) |\mathbf{R}_A - \mathbf{R}_B|^2 \exp \left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j}|\mathbf{R}_A - \mathbf{R}_B|^2 \right]}
\end{align*}
$$

In [9]:
def f1AB(d:np.array, a:np.array, RA:np.array, RB:np.array):
    """
        Término f_1^AB. Suma de funciones base
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    (RA, RB) : coord. núcleo (A,B) -> np.array
    """
    RAB2 = np.square(np.linalg.norm(RA-RB)) # distancia interatómica al cuadrado, |RA-RB|^2
    
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            p = a[i]+a[j] # exponente total
            mu = a[i]*a[j]/p # exponente reducido
            suma += d[i]*d[j]*np.power(a[i]*a[j], 1.75) / np.power(p, 3.5) * (3-2*a[i]*a[j]) * RAB2 * np.exp(-mu*RAB2)
    return np.power(2, 1.5)/np.pi * suma

<div class='alert alert-block alert-info'><strong>Término $f_{2}^{(AB)}$</strong></div>

Para $f_{2}^{(AB)}$ se tiene

$$
f_{2}^{(AB)} = (\varphi_A | \hat{f}_2 | \varphi_B)
= \int \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \left( - \sum_C \frac{Z_C}{r_C} \right) \left( \sum_j^{k} d_j \phi_{j}^{(B)} \right) \, \mathrm{d}\mathbf{r}\\
= - \sum_i^{k} \sum_j^{k} d_i d_j \Bigg( \sum_C Z_C \underbrace{\int \frac{\phi_{i}^{(A)} \phi_{j}^{(B)}}{r_C} \, \mathrm{d}\mathbf{r}}_{I_2^{AB}} \, \Bigg)
$$

Para la multiplicación de Gaussianas $\phi^{(A)} \phi^{(B)}$ de la integral $I_2^{AB}$ considere el resultado [2, pág. 341]:

$$ \phi^{(A)} \phi^{(B)} = \exp(-\alpha |\mathbf{r} - \mathbf{R}_A|^2) \exp(-\beta |\mathbf{r} - \mathbf{R}_B|^2)
= \tilde{K}_{AB} \exp(-p|\mathbf{r} - \mathbf{R}_P|^2)
= \tilde{K}_{AB} \exp(-p \, r_P^2)
$$

con

$$\tilde{K}_{AB} = \exp\Big( - \frac{\alpha \beta}{\alpha + \beta} |\mathbf{R}_A - \mathbf{R}_B|^2 \Big)\,, \qquad
p = \alpha + \beta \,, \qquad
\mathbf{R}_P = \frac{\alpha \mathbf{R}_A + \beta \, \mathbf{R}_B}{\alpha + \beta} $$

donde $\tilde{K}_{AB}$ es el factor pre-exponencial, $p$ es el exponente total y $\mathbf{R}_P$ es la coordenada del centro de carga.

In [10]:
def K(a:float, b:float, RA:np.array, RB:np.array):
    """
        Factor pre-exponencial
    (a, b) : exponente orbital Gaussiano de la función centrada en (A, B)
    (RA, RB) : coordenadas del núcleo (A, B)
    """
    RAB2 = np.square(np.linalg.norm(RA-RB)) # distancia interatómica al cuadrado, |RA - RB|^2
    return np.exp(-(a*b) / (a+b) * RAB2)

In [11]:
def RP(a:float, b:float, RA:np.array, RB:np.array):
    """
        Coordenada del centro de carga RP
    (a, b) : exponente orbital Gaussiano de la función centrada en (A, B)
    (RA, RB) : coordenadas del núcleo (A, B)
    """
    return (a*RA + b*RB)/(a+b)

Para el término $1/r_C$ de la integral $I_2^{AB}$ considere que [2, pág.344]

$$ \int_{-\infty}^{\infty} \exp(-a x^2) \, \mathrm{d}x = \sqrt{\frac{\pi}{a}} \,, \quad \alpha \in \mathbb{R}_{> 0} $$

de lo cual

$$ \frac{1}{r_C} = \frac{1}{\sqrt{\pi}} \int_{-\infty}^{\infty} \exp(-r_C^2 t^2) \, \mathrm{d}t $$

Del trabajo previo se tendrá que la integral $I_2^{AB}$ es

$$ I_2^{AB} = \tilde{K}_{AB} \int \exp(-p \, r_P^2) \left[ \frac{1}{\sqrt{\pi}} \int_{-\infty}^{\infty} \exp(-r_C^2 t^2) \, \mathrm{d}t \right] \, \mathrm{d}\mathbf{r} $$

Se multiplican ambas Gaussianas [2, pág341] similarmente a como se hizo previamente con $\phi^{A}$ y $\phi^{B}$:

$$ \exp(-p |\mathbf{r} - \mathbf{R}_P|^2) \exp(-t^2 |\mathbf{r} - \mathbf{R}_C|^2)
= \tilde{K}_{PC} \exp(- (p + t^2) |\mathbf{r} - \mathbf{R}_S|^2)
= \tilde{K}_{PC} \exp(- (p + t^2) \, r_S^2)
$$

donde

$$ \tilde{K}_{PC} = \exp \left( - \frac{p \, t^2}{p + t^2} R_{PC}^2 \right)\,, \qquad
\mathbf{R}_S = \frac{p \mathbf{R}_P + t^2 \mathbf{R}_C}{p + t^2} $$

con $R_{PC} = |\mathbf{R}_P - \mathbf{R}_C|$ y $\mathbf{R}_S$ es un punto en la línea que conecta a $\mathbf{R}_P$ y $\mathbf{R}_C$.

Así pues, la integral $I_2^{AB}$ es entonces

$$ I_2^{AB} = \frac{\tilde{K}_{AB}}{\sqrt{\pi}} \int_{-\infty}^{\infty} \Bigg\{ \underbrace{\int \exp[- (p + t^2) \, r_S^2] \, \mathrm{d}\mathbf{r}}_{I_S} \Bigg\} \exp \left( - \frac{p \, t^2}{p + t^2} R_{PC}^2 \right) \, \mathrm{d}t $$

<div class='alert alert-block alert-warning'>
Helgaker evalúa a la integral $I_{S}$ sobre todo el espacio [2, pág.362], es decir considera $\mathrm{d}\mathbf{r} = r^2 \sin \theta \, \mathrm{d}r \, \mathrm{d}\theta \, \mathrm{d}\phi$. Para el presente trabajo ya se integró sobre las coordenadas angulares lo cual resultaba en un factor de 1 debido al armónico esférico $Y_{00}$.
</div>

Por lo discutido previamente, aquí $\mathrm{d}\mathbf{r} = r^2 \mathrm{d}r$ en cuyo caso la integral $I_S$ será

$$ I_S = \frac{\sqrt{\pi}}{4 \, (p + t^2)^{3/2}} $$

Mas aún, fíjese $I_S$ y $\exp \left( - \frac{p \, t^2}{p + t^2} R_{PC}^2 \right)$ son funciones pares, así puede tomarse dos veces la integral respecto a $t$ con límites de integración de 0 a $\infty$. La integral $I_{2}^{AB}$ se reduce a

$$ I_2^{AB}
= \tfrac{1}{2} \tilde{K}_{AB} \int_{0}^{\infty} (p + t^2)^{-3/2} \exp \left( - p \, R_{PC}^2 \frac{t^2}{p + t^2} \right) \, \mathrm{d}t
$$

Haciendo el cambio de variable $u^2 = t^2/(p + t^2)$ que lleva a $\mathrm{d}t = \sqrt{p}(1-u^2)^{-3/2}$ se sigue que

$$ I_2^{AB} = \frac{\tilde{K}_{AB}}{2 p} \int_{0}^{1} \exp \left( - p \, R_{PC}^2 \, u^2 \right) \, \mathrm{d}u $$

Esta última integral pertenece a una clase de funciones conocidas por *funciones de Boys* y son de la forma
$$ F_n(x) = \int_0^1 \exp(-x t^2) t^{2n} \, \mathrm{d}t $$

Función la cual está relacionada con las función Gamma $\Gamma(n)$ y la función Gamma inferior incompleta $P(n,x)$ [2, pág.368]:

$$ F_n(x) = \frac{\Gamma(n+\frac{1}{2}) P(n+\frac{1}{2},x)}{2 x^{n + 1/2}}
$$

donde

$$ \Gamma(n) = \int_{0}^{\infty} \exp(-t) t^{n-1} \, \mathrm{d}t\,, \qquad
P(n,x) = \frac{1}{\Gamma(n)} \int_{0}^{x} \exp(-t) t^{n-1} \, \mathrm{d}t
$$

Así, el valor final de la integral buscada es

$$ \boxed{I_2^{AB} = \frac{\tilde{K}_{AB}}{2 p} F_{0}(p \, R_{PC}^2)} $$

In [12]:
def Boys(n:float, x:float):
    """
        Función de Boys Fn(x)
    n : orden de la función
    x : argumento
    """
    return factorial(n+1/2) * gammainc(n,x) / (2 * np.power(x, n+1/2))

Finalmente, para $f_{2}^{(AB)}$ se tiene entonces que

$$ f_{2}^{(AB)}
= -\sum_i^{k} \sum_j^{k} d_i d_j \Bigg\{ \sum_C Z_C \Bigg[ \frac{\tilde{K}_{AB}}{2 p} F_{0}(p \, R_{PC}^2) \Bigg] \Bigg\}
= -\frac{1}{2} \sum_i^{k} \sum_j^{k} \left\{ \frac{d_i d_j}{\alpha_i + \alpha_j} \tilde{K}_{AB} \Bigg[ \sum_C Z_C F_{0}((\alpha_i + \alpha_j) \, R_{PC}^2) \Bigg] \right\}
$$

donde

$$ \tilde{K}_{AB} = \exp\Big( - \frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \Big)\,, \qquad
R_{PC} = |\mathbf{R}_P - \mathbf{R}_C|\,, \qquad
\mathbf{R}_P = \frac{\alpha_i \mathbf{R}_A + \alpha_j \mathbf{R}_B}{\alpha_i + \alpha_j}
$$

Incluyendo los factores de normalización se obtiene:

$$ \begin{align*}
f_{2}^{(AB)}
&= -\frac{1}{2} \sum_i^{k} \sum_j^{k} \left\{ \left( \frac{2(2\alpha_i)^{3/4}}{\pi^{1/4}} \right) \left( \frac{2(2\alpha_j)^{3/4}}{\pi^{1/4}} \right) \frac{d_i d_j}{\alpha_i + \alpha_j} \tilde{K}_{AB} \Bigg[ \sum_C Z_C F_{0}((\alpha_i + \alpha_j) \, R_{PC}^2) \Bigg] \right\} \\
&= \boxed{- \frac{2^{5/2}}{\sqrt{\pi}} \sum_i^{k} \sum_j^{k} d_i d_j \frac{(\alpha_i \alpha_j)^{3/4}}{\alpha_i + \alpha_j} \tilde{K}_{AB} \Bigg[ \sum_C Z_C F_{0}((\alpha_i + \alpha_j) \, R_{PC}^2) \Bigg] }
\end{align*}
$$

In [21]:
def Z2AB(ai:float, aj:float, R:list, Z:np.array, RP:np.array):
    """
        Suma sobre los núcleos A y B
    (ai, aj) : exponente orbital Gaussiano de la función centrada en (A, B)
    R : vector de núcleos [Ra, Rb] donde Rc = [C1, C2, C3]
    Z : vector de cargas atómicas [Za, Zb]
    RP : coordenada del centro de carga
    """
    suma = 0
    for C in range(len(Z)):
        RPC2 = np.square(np.linalg.norm(RP-R[C])) # distancia interatómica al cuadrado, |RP-RC|^2
        suma += Z[C] * Boys(0, (ai+aj)*RPC2)
    return suma

In [23]:
def f2AB(d:np.array, a:np.array, RA:float, RB:float, ZA:float, ZB:float):
    """
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    (RA, RB) : coord. núcleo (A, B)
    (ZA, ZB) : carga nuclear del núcleo (A, B)
    """
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            KAB = K(a[i], a[j], RA, RB) # factor pre-exponencial
            Rp = RP(a[i], a[j], RA, RB) # coordenada del centro de carga
            suma += d[i]*d[j] *np.power(a[i]*a[j], 0.75) / (a[i]+a[j]) * KAB * Z2AB(a[i], a[j], [RA, RB], [ZA, ZB], Rp)
    return -np.power(2,2.5)/np.sqrt(np.pi) * suma

<div class='alert alert-block alert-info'><strong>Término $f_{2}^{(AA)}$</strong></div>

Para $f_{2}^{(AA)}$ se tiene

$$
f_{2}^{(AB)} = (\varphi_A | \hat{f}_2 | \varphi_A)
= \sum_i^{k} \sum_j^{k} d_i d_j \Big( - \sum_C Z_C \int \frac{\phi_{i}^{(A)} \phi_{j}^{(A)}}{r_C} \, \mathrm{d}\mathbf{r} \Big)
= \sum_i^{k} \sum_j^{k} d_i d_j \Bigg( - Z_A \underbrace{\int \frac{\phi_{i}^{(A)} \phi_{j}^{(A)}}{r_A} \, \mathrm{d}\mathbf{r}}_{I_1} - Z_B \underbrace{\int \frac{\phi_{i}^{(A)} \phi_{j}^{(A)}}{r_B} \, \mathrm{d}\mathbf{r}}_{I_2} \Bigg) \,.
$$

La multiplicación de Gaussianas $\phi_i^{(A)} \phi_j^{(A)}$ no es más que:

$$ \phi_i^{(A)} \phi_j^{(A)} = \exp(-\alpha_i |\mathbf{r} - \mathbf{R}_A|^2) \exp(-\alpha_i |\mathbf{r} - \mathbf{R}_A|^2)
= \exp[-(\alpha_i + \alpha_j) |\mathbf{r} - \mathbf{R}_A|^2]
= \exp[-(\alpha_i + \alpha_j) r_A^2] \,.
$$

Para la integral $I_1$ considere el siguiente desarrollo:

$$ I_1
= \int \frac{\exp[-(\alpha_i + \alpha_j) r_A^2]}{r_A} \, \mathrm{d}\mathbf{r}
= \int_0^\infty r \exp[-(\alpha_i + \alpha_j) r^2] \, \mathrm{d}r
= \frac{1}{2(\alpha_i + \alpha_j)} \,.
$$

Observe que $I_2$ tiene la misma estructura que $I_2^{(AB)}$ desarrollado para el caso del término $f_2^{(AB)}$. No obstante, para este caso se tiene que $\tilde{K}_{AB} = 1$, $\mathbf{R}_P = \mathbf{R}_A$ y $\mathbf{R}_C = \mathbf{R}_B$, de lo cual

$$ I_2
= \frac{1}{2 p} F_{0}(p \, R_{AB}^2)
= \frac{1}{2 (\alpha_i + \alpha_j)} F_{0}\big[ (\alpha_i + \alpha_j) \, R_{AB}^2 \big] \,.
$$

Finalmente, el término $f_2^{(AA)}$ es

$$ f_{2}^{(AA)}
= - \tfrac{1}{2} \sum_i^{k} \sum_j^{k} \frac{d_i d_j}{\alpha_i + \alpha_j} \Big\{ Z_A + Z_B F_0 \big[(\alpha_i + \alpha_j) R_{AB}^2 \big] \Big\}
$$

In [24]:
def f2AA(d:np.array, a:np.array, RA:np.array, RB:np.array, ZA:float, ZB:float):
    """
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    (RA, RB) : coord. núcleo (A, B)
    (ZA, ZB) : carga nuclear del núcleo (A, B)
    """
    RAB2 = np.square(np.linalg.norm(RA-RB)) # distancia interatómica al cuadrado, |RA-RB|^2
    
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            # const(a) = const. de normalización de Gaussianas 1s
            suma += const(a[i]) * const(a[j]) * d[i]*d[j]/(a[i]+a[j]) * ( ZA + ZB*Boys(0, (a[i]+a[j])*RAB2) )
    
    return (-1/2) * suma

<div class='alert alert-block alert-success'><strong>Elemento de matriz $f_{pp}$</strong></div>

Finalmente, es posible calcular el elemento de matriz $f_{pp} = 2 \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right]$. Si $p=1,2$ entonces el signo es positivo, mientras que si $p=3,4$ el signo será negativo.

In [25]:
def fpp(p:int, d:np.array, a:np.array, RA:np.array, RB:np.array, ZA:float, ZB:float):
    """
        Elemento de matriz f_pp
    p : p-ésimo elemento de la base ordenada
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    (RA, RB) : coord. núcleo (A,B)
    (ZA, ZB) : carga nuclear, núcleo (A, B)
    """
    term1AA = f1AA(d, a)
    term1AB = f1AB(d, a, RA, RB)
    term2AA = f2AA(d, a, RA, RB, ZA, ZB)
    term2AB = f2AB(d, a, RA, RB, ZA, ZB)

    if p in [1,2]:
        return 2*(term1AA + term1AB + term2AA + term2AB)
    else:
        return 2*(term1AA - term1AB + term2AA - term2AB)

In [93]:
# Parámetros Gaussianos obtenidos en `(1) STO-3G.ipynb`
d = np.array([0.44471812476789035, 0.5352716544572346, 0.1543000507808527])
a = np.array([0.10983311305458726, 0.40584959263313375, 2.227949029647934])

RA = np.array([0, 0, 0])
RB = np.array([1.401, 0, 0])
ZA = 1
ZB = 1

f11 = fpp(1, d, a, RA, RB, ZA, ZB)
f33 = fpp(3, d, a, RA, RB, ZA, ZB)

In [94]:
f11

np.float64(-4.292930320257096)

In [95]:
f33

np.float64(0.885745439483749)

In [96]:
# revisión: f1AB = f1BA
print('f1AB:', f1AB(d, a, RA, RB))
print('f1BA:', f1AB(d, a, RB, RA))

f1AB: 0.25782165875825447
f1BA: 0.25782165875825447


In [97]:
# revisión: f2AA = f2BB
print('f2AA:', f2AA(d, a, RA, RB, ZA, ZB))
print('f2BB:', f2AA(d, a, RB, RA, ZA, ZB))

f2AA: -1.3461013356356974
f2BB: -1.3461013356356974


In [98]:
# revisión: f2AA = f2BB
print('f2AB:', f2AB(d, a, RA, RB, ZA, ZB))
print('f2BA:', f2AB(d, a, RB, RA, ZA, ZB))

f2AB: -1.5524905986934656
f2BA: -1.5524905986934658


In [ ]:
def f2AB_v2(d:np.array, a:np.array, RA:float, RB:float, ZA:float, ZB:float):
    """
        Modern quantum chemistry
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    (RA, RB) : coord. núcleo (A, B)
    (ZA, ZB) : carga nuclear del núcleo (A, B)
    """
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma +=
    return suma